# Multi-Step Reasoning with LLM Chains

This notebook demonstrates **multi-step reasoning** using hexDAG:

- **Parallel analysis** - Multiple perspectives analyzed concurrently
- **Context synthesis** - Results combined into final conclusion
- **YAML-first approach** - Declarative pipeline definition

## Architecture

```
Step 0: LLM (initial analysis)    ─┐
                                   ├─→ Synthesis Function → Final Output
Step 1: LLM (historical context) ─┘
```

Parallel LLM nodes feed into a synthesis function that combines their outputs.

## Setup

We use a **MockLLM** to simulate multi-step reasoning responses. This approach:
- Ensures reproducible demos
- Works offline without API keys
- Demonstrates the architecture clearly

In [ ]:
from hexdag.stdlib.adapters.mock.mock_llm import MockLLM

# Create mock LLM with responses for each parallel analysis
mock_llm = MockLLM(
    responses=[
        # Initial analysis response (for initial_analysis node)
        """**Initial Analysis of Artificial Intelligence**

AI refers to the simulation of human intelligence by machines.
Key areas: Machine Learning, NLP, Computer Vision, Robotics.""",
        # Historical context response (for historical_analysis node)
        """**Historical Context**

- 1950: Turing Test proposed
- 1956: "AI" coined at Dartmouth
- 1997: Deep Blue beats Kasparov
- 2023: LLMs achieve remarkable capabilities""",
    ]
)

print("✅ MockLLM configured with 2 responses for parallel analysis")

## Define Synthesis Function

The synthesis function combines outputs from parallel analysis nodes.

In [ ]:
# Synthesis function receives outputs from dependency nodes and produces final result
async def synthesize_and_conclude(inputs: dict) -> str:
    """Combine parallel analyses into final conclusion.

    When a node depends on multiple nodes, it receives their outputs as a dict
    with node names as keys.
    """
    initial = inputs.get("initial_analysis", "")
    historical = inputs.get("historical_analysis", "")

    return f"""━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
MULTI-STEP REASONING RESULTS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📊 INITIAL ANALYSIS:
{initial}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📜 HISTORICAL CONTEXT:
{historical}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎯 FINAL CONCLUSION:
Artificial Intelligence represents a transformative technology spanning 
70+ years of development. From the Turing Test to modern LLMs, AI has 
evolved from theoretical concepts to practical applications across 
healthcare, transportation, and communication.

Key insights:
• AI encompasses ML, NLP, computer vision, and robotics
• Evolution from rule-based systems to deep learning
• Powered by large-scale data and compute resources
• Raising important ethical considerations

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"""


print("✅ Synthesis function defined")

## Define Pipeline

Two parallel LLM nodes feed into a synthesis function, then to final LLM.

In [ ]:
# Multi-step reasoning pipeline: parallel LLM → synthesis function
pipeline_yaml = """
apiVersion: hexdag/v1
kind: Pipeline
metadata:
  name: multi-step-reasoning
  description: Parallel analysis with synthesis

spec:
  nodes:
    # Parallel analysis nodes (run concurrently)
    - kind: llm_node
      metadata:
        name: initial_analysis
      spec:
        prompt_template: |
          Analyze the topic: {{topic}}
          Provide an initial analysis covering key concepts.
      depends_on: []

    - kind: llm_node
      metadata:
        name: historical_analysis
      spec:
        prompt_template: |
          Provide historical context for: {{topic}}
          Cover key milestones and developments.
      depends_on: []

    # Final synthesis combines all parallel outputs
    - kind: function_node
      metadata:
        name: final_conclusion
      spec:
        fn: __main__.synthesize_and_conclude
      depends_on: [initial_analysis, historical_analysis]
"""

print("Pipeline: 2 parallel LLM nodes → synthesis function")

## Execute Pipeline

In [ ]:
from hexdag.compiler import YamlPipelineBuilder
from hexdag.kernel.orchestration.orchestrator import Orchestrator

# Build and execute
builder = YamlPipelineBuilder()
graph, config = builder.build_from_yaml_string(pipeline_yaml)

print(f"✅ Pipeline built: {len(graph.nodes)} nodes")
print(f"   Nodes: {list(graph.nodes.keys())}")

orchestrator = Orchestrator(ports={"llm": mock_llm})
results = await orchestrator.run(graph, {"topic": "Artificial Intelligence"})

print("\n✅ Execution complete!")

In [ ]:
# Display results
print("=" * 60)
print("FINAL CONCLUSION")
print("=" * 60)
print(results["final_conclusion"])
print("\n✅ Multi-step reasoning complete!")